# HLA Locus-Specific Embeddings Analysis

This notebook demonstrates how to use the locus-specific embeddings generated by the `analyze_locus_embeddings.py` script and how to interpret the visualizations.

## Setup

First, let's set up our environment and import the necessary libraries.

In [ ]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from IPython.display import Image, display, Markdown

# Add parent directory to path to import modules
notebook_dir = Path().resolve()
project_dir = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
sys.path.insert(0, str(project_dir))

# Import our modules
from src.models.protbert import ProtBERTEncoder
from src.analysis.visualization import HLAEmbeddingVisualizer
from src.utils.logging import setup_logging

# Set up logging
logger = setup_logging(level="INFO")

# Set paths
data_dir = project_dir / "data"
sequence_file = data_dir / "processed" / "hla_sequences.pkl"
embeddings_dir = data_dir / "embeddings"
analysis_dir = data_dir / "analysis" / "locus_embeddings"

print(f"Project directory: {project_dir}")
print(f"Sequence file: {sequence_file}")
print(f"Analysis directory: {analysis_dir}")

## Check Generated Visualizations

Let's check the visualizations that were generated by the script for each locus. We'll look at both Class I and Class II loci.

In [ ]:
def display_locus_visualizations(locus, class_type):
    """Display all visualizations for a specific locus"""
    plots_dir = analysis_dir / class_type / "plots"
    
    # Check if directory exists
    if not plots_dir.exists():
        print(f"Directory {plots_dir} does not exist. Run the analysis script first.")
        return
    
    # Display header
    display(Markdown(f"### HLA-{locus} Visualizations"))
    
    # Display each visualization type
    for viz_type in ["umap", "tsne", "pca", "groups"]:
        plot_file = plots_dir / f"hla_{locus}_{viz_type}.png"
        if plot_file.exists():
            display(Markdown(f"#### {viz_type.upper()} Projection"))
            display(Image(filename=str(plot_file)))
        else:
            print(f"Plot {plot_file} not found")
    
    print("\n")

### Class I HLA Loci

Let's examine the visualizations for Class I HLA loci (A, B, C).

In [ ]:
for locus in ["A", "B", "C"]:
    display_locus_visualizations(locus, "class1")

### Class II HLA Loci

Now let's examine the visualizations for Class II HLA loci (DRB1, DQB1, DPB1).

In [ ]:
for locus in ["DRB1", "DQB1", "DPB1"]:
    display_locus_visualizations(locus, "class2")

## Load and Examine Embeddings

Let's load the embeddings for a specific locus and examine them in more detail.

In [ ]:
def load_locus_embeddings(locus, class_type):
    """Load embeddings for a specific locus"""
    embeddings_file = analysis_dir / class_type / "embeddings" / f"hla_{locus}_embeddings.pkl"
    
    if not embeddings_file.exists():
        print(f"Embeddings file {embeddings_file} not found. Run the analysis script first.")
        return None
    
    with open(embeddings_file, 'rb') as f:
        embeddings = pickle.load(f)
    
    print(f"Loaded {len(embeddings)} embeddings for HLA-{locus}")
    return embeddings

In [ ]:
# Let's examine HLA-A embeddings as an example
locus = "A"
embeddings = load_locus_embeddings(locus, "class1")

if embeddings is not None:
    # Show some basic stats
    alleles = list(embeddings.keys())
    sample_allele = alleles[0]
    embedding_dim = embeddings[sample_allele].shape[0]
    
    print(f"Embedding dimension: {embedding_dim}")
    print(f"Sample alleles: {', '.join(alleles[:5])}")
    
    # Show a sample embedding
    print(f"\nSample embedding for {sample_allele}:")
    print(embeddings[sample_allele][:10])  # Show just the first 10 dimensions

## Custom Similarity Analysis

Let's perform a custom analysis to find similar alleles across different loci.

In [ ]:
# Initialize encoder (needed for the visualizer)
encoder = ProtBERTEncoder(
    sequence_file=sequence_file,
    cache_dir=embeddings_dir
)

# Initialize visualizer
visualizer = HLAEmbeddingVisualizer(encoder)

In [ ]:
def cosine_similarity(a, b):
    """Compute cosine similarity between two vectors"""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def find_similar_alleles_across_loci(query_allele, query_embeddings, target_embeddings, top_k=5):
    """Find most similar alleles in the target embeddings"""
    if query_allele not in query_embeddings:
        print(f"Allele {query_allele} not found in embeddings")
        return []
    
    query_embedding = query_embeddings[query_allele]
    similarities = []
    
    for target_allele, target_embedding in target_embeddings.items():
        similarity = cosine_similarity(query_embedding, target_embedding)
        similarities.append((target_allele, similarity))
    
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_k]

In [ ]:
# Load embeddings for different loci
locus_a_embeddings = load_locus_embeddings("A", "class1")
locus_b_embeddings = load_locus_embeddings("B", "class1")

# Choose a query allele
query_allele = "A*02:01"  # A common HLA-A allele

# Find similar HLA-B alleles
if locus_a_embeddings is not None and locus_b_embeddings is not None:
    similar_alleles = find_similar_alleles_across_loci(
        query_allele, locus_a_embeddings, locus_b_embeddings, top_k=10
    )
    
    print(f"HLA-B alleles most similar to {query_allele}:")
    for allele, similarity in similar_alleles:
        print(f"  {allele}: similarity={similarity:.4f}")

## Custom Visualization

Now let's create a custom visualization that combines alleles from different loci.

In [ ]:
def create_combined_visualization(embeddings_dict, method="umap", title=None):
    """Create a visualization that combines embeddings from different loci"""
    # Combine all embeddings
    combined_embeddings = {}
    for locus, embeddings in embeddings_dict.items():
        if embeddings is not None:
            combined_embeddings.update(embeddings)
    
    # Create visualization
    if combined_embeddings:
        return visualizer.visualize_embeddings(
            combined_embeddings,
            method=method,
            color_by="locus",
            title=title or f"Combined HLA Embeddings - {method.upper()} Projection",
            n_neighbors=20,
            min_dist=0.1,
            random_state=42
        )
    else:
        print("No embeddings available for visualization")
        return None

In [ ]:
# Load embeddings for Class I loci
embeddings_dict = {
    "A": load_locus_embeddings("A", "class1"),
    "B": load_locus_embeddings("B", "class1"),
    "C": load_locus_embeddings("C", "class1")
}

# Create combined visualization
plt.figure(figsize=(12, 10))
create_combined_visualization(embeddings_dict, method="umap", title="HLA Class I Embeddings - UMAP Projection")
plt.tight_layout()
plt.show()

## Interpreting the Visualizations

### What to Look For

1. **Clustering patterns**: Alleles that are functionally similar should cluster together in the embedding space.
2. **Locus separation**: Different loci (A, B, C, etc.) should generally form distinct clusters.
3. **Protein group similarity**: Alleles with the same first field (e.g., A*01) should cluster together.
4. **Outliers**: Look for alleles that appear isolated from their expected clusters, which might indicate unusual sequences.

### Different Dimensionality Reduction Techniques

- **UMAP**: Best preserves the local structure of the data, showing stronger clustering patterns.
- **t-SNE**: Similar to UMAP but sometimes shows slightly different cluster formations.
- **PCA**: Preserves global structure better but may not show distinct clusters as clearly.

### Applications

These visualizations can be used for:
1. Understanding the relationship between different HLA alleles
2. Identifying functionally similar alleles across different nomenclature
3. Finding representative alleles for broader allele groups
4. Improving donor-recipient matching by identifying functionally similar substitutions

## Conclusion

In this notebook, we've demonstrated how to:

1. Load and visualize the locus-specific embeddings generated by the analysis script
2. Examine the properties of the embeddings
3. Perform cross-locus similarity analysis
4. Create custom visualizations that combine multiple loci
5. Interpret the different visualization techniques

These embeddings and visualizations provide valuable insights into the relationships between HLA alleles, which can be used for a variety of research and clinical applications.